In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import pyaudio
import playsound

In [3]:
data_train = pd.read_csv("../../data/data_train.csv")
data_test = pd.read_csv("../../data/data_test.csv")